In [1]:
# Random Forest Algorithm
# seeding the generated number makes our results reproducible (good for debugging)
# This module implements pseudo-random number generators for various distributions.
from csv import reader
from datetime import datetime
from math import sqrt
from matplotlib import pyplot
from random import randrange
from random import seed
from sklearn import preprocessing
from sklearn import svm, datasets
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.metrics import auc
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import numpy as np
import numpy as npX
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv("train.csv")
df.head()

,tweet_id,priority,category,event_id,identifier,description,id,desc,level,intentType,exampleLowLevelTypes,eventID,eventName,eventDescription,eventType,postID,postCategories,postPriority
0,211565974422425600,0.75,ServiceAvailable,19.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-OriginalEvent,A report of the original event occuring. This ...,High-level,Report,[],fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211565974422425600,['ServiceAvailable'],High
1,211654415503990784,0.50,News,15.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-Official,An official report by a government or public s...,High-level,Report,"['OfficialStatement', 'RegionalWarning', 'Publ...",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211654415503990784,['News'],Medium
2,211681309368655872,0.25,News,15.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-Official,An official report by a government or public s...,High-level,Report,"['OfficialStatement', 'RegionalWarning', 'Publ...",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211681309368655872,['News'],Low
3,211685621125742592,0.25,Official,16.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-News,The post is a news report providing/linking to...,High-level,Other,"['NewsHeadline', 'NewsArticle']",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211685621125742592,['Official'],Low
4,211877049147736064,0.25,Factoid,7.0,TRECIS-ITR-H-Types-v4-T1,TREC-IS Incident Tweet Routing (High-Level) In...,Report-ThirdPartyObservation,The user is reporting a information that they ...,High-level,Report,"['Group/IndividualMovement', 'PeopleEvacuating...",fireColorado2012,DEMO: CrisisLex26 2012 Colorado wildfires,The 2012 Colorado wildfires were an unusually ...,Unknown,211877049147736064,['Factoid'],Low


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tweet_id              943 non-null    int64  
 1   priority              943 non-null    float64
 2   category              943 non-null    object 
 3   event_id              922 non-null    float64
 4   identifier            922 non-null    object 
 5   description           922 non-null    object 
 6   id                    922 non-null    object 
 7   desc                  922 non-null    object 
 8   level                 922 non-null    object 
 9   intentType            922 non-null    object 
 10  exampleLowLevelTypes  922 non-null    object 
 11  eventID               943 non-null    object 
 12  eventName             943 non-null    object 
 13  eventDescription      943 non-null    object 
 14  eventType             943 non-null    object 
 15  postID                9

In [4]:
X2 = df[['tweet_id', 'priority', 'eventID','intentType']]
y2 = df['tweet_id'].truncate(after=942)

X2.shape

(943, 4)

In [5]:
### Split the dataset into 'Train' and 'Test' sets
X_train, X_test, y_train, y_test = train_test_split(X2, y2, random_state=0)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (707, 4)
X_test shape: (236, 4)
y_train shape: (707,)
y_test shape: (236,)


In [6]:
numeric_features = ['tweet_id', 'priority']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['eventID','intentType']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.000


In [7]:
from sklearn import set_config
set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tweet_id', 'priority']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['eventID', 'intentType'])])),
                ('classifier', LogisticRegression())])